In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
def get_featuers_by_name(net_name):
    features_train = np.load('./features/' + net_name + '/features_train.npy')
    features_dev = np.load('./features/'+ net_name +'/features_dev.npy')
    return features_train, features_dev

def get_features(names):
    features_train = []
    features_dev = []
    for name in names:
        train, dev = get_featuers_by_name(name)
        features_train.append(train)
        features_dev.append(dev)
    
    features_train = np.concatenate(features_train, axis=1)
    features_dev = np.concatenate(features_dev, axis=1)
    return np.stack([features_train, features_train], axis=1), np.stack([features_dev, features_dev], axis=1)

def get_classes_by_name(name):
    classes_train = np.load('./features/res_net/classes_train_' + name + '.npy')
    filenames_train = np.load('./features/res_net/filenames_train.npy')
    classes_train = to_categorical(classes_train)
    classes_dev = np.load('./features/res_net/classes_dev_' + name + '.npy')
    filenames_dev = np.load('./features/res_net/filenames_dev.npy')
    classes_dev = to_categorical(classes_dev)
    return classes_train, classes_dev, filenames_train, filenames_dev

def get_classes(names):
    classes_train = []
    classes_dev = []
    num_classes = []
    for name in names:
        train, dev, filenames_train, filenames_dev = get_classes_by_name(name)
        classes_train.append(train)
        classes_dev.append(dev)
        num_classes.append(train.shape[1])
        
    max_num_classes = np.max(num_classes)
    
    padded_classes_train = []
    padded_classes_dev = []
    for train, dev in zip(classes_train, classes_dev):
        padded_classes_train.append(
            np.pad(
                train, 
                pad_width=((0, 0), (0, max_num_classes - train.shape[1])),
                mode='constant',
                constant_values=((0,0), (0,0)),
            )
        )
        
        padded_classes_dev.append(
            np.pad(
                dev, 
                pad_width=((0, 0), (0, max_num_classes - dev.shape[1])),
                mode='constant',
                constant_values=((0,0), (0,0)),
            )
        )
        
    return np.stack(padded_classes_train, axis=1), np.stack(padded_classes_dev, axis=1), filenames_train, filenames_dev

def shuffle(data, classes):
    assert data.shape[0] == classes.shape[0]
    indices = np.random.permutation(data.shape[0])
    return data[indices], classes[indices]

def preprocess(path, nets, names, batch_size, save=True):
    features_train, features_dev = get_features(nets)
    classes_train, classes_dev, _, __ = get_classes(names)
    
    features_train, classes_train = shuffle(features_train, classes_train)
    features_dev, classes_dev = shuffle(features_dev, classes_dev)
    
    if save != True:
        return

    for i in range(features_train.shape[0] // batch_size + 1):
        np.save(
            path + str(len(names)) + '_steps' + '/features_train_' + str(i) + '.npy', 
            features_train[i * batch_size : i * batch_size + batch_size, :, :],
        )
        
        np.save(
            path + str(len(names)) + '_steps' + '/classes_train_' + str(i) + '.npy', 
            classes_train[i * batch_size : i * batch_size + batch_size, :, :],
        )
    print('Train shape: ', features_train.shape, 'Dev shape:', features_dev.shape)
        
    for i in range(features_dev.shape[0] // batch_size + 1):
        np.save(
            path + str(len(names)) + '_steps' + '/features_dev_' + str(i) + '.npy', 
            features_dev[i * batch_size : i * batch_size + batch_size, :, :]
        )
        np.save(
            path + str(len(names)) + '_steps' + '/classes_dev_' + str(i) +'.npy', 
            classes_dev[i * batch_size : i * batch_size + batch_size, :, :],
        )

    print('Train classes shape: ', classes_train.shape, 'Dev classes shape:', classes_dev.shape)

In [3]:
preprocess(
    './features/lstm/',
    [
        'res_net',
        'inception',
        'xception',
    ],
    [
        'make', 
        'model',
    ],
    128,
    #save=False,
)

[ 0  0  0 ..., 48 48 48]
['AM Hummer SUV 2000/00163.jpg' 'AM Hummer SUV 2000/00462.jpg'
 'AM Hummer SUV 2000/00887.jpg' ...,
 'smart fortwo Convertible 2012/06886.jpg'
 'smart fortwo Convertible 2012/07202.jpg'
 'smart fortwo Convertible 2012/07236.jpg']
[ 77  77  77 ..., 165 165 165]
['AM Hummer SUV 2000/00163.jpg' 'AM Hummer SUV 2000/00462.jpg'
 'AM Hummer SUV 2000/00887.jpg' ...,
 'smart fortwo Convertible 2012/06886.jpg'
 'smart fortwo Convertible 2012/07202.jpg'
 'smart fortwo Convertible 2012/07236.jpg']
